In [77]:
import sys
sys.path.append('..')

import os
from pathlib import Path

import re
import nltk 
# nltk.download('punkt')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download("stopwords")

# from pymystem3 import Mystem
from string import punctuation

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = (20,10)

import pickle
import numpy as np 

from src.embedders import LASEREmbedder


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sentenced210/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
with open('data.pickle', 'rb') as f:
    data_dict = pickle.load(f)

print(data_dict.keys())

dict_keys(['sentences_embeddings', 'sentences', 'entities', 'sentences_without_entities'])


In [3]:
print(len(data_dict['sentences']))

3263


In [4]:
sentences = []
for e in data_dict['sentences']:
    sentences.append(' '.join(e))

In [5]:
print(sentences)


['Премьер-министр Японии Наото Кан огласил новый состав правительства страны .', 'Так , бывшего генерального секретаря правительства Йошито Сенгоку сменил на его посту Юкио Эдано .', "Он также заверил , что в случае , если Мубарак уйдет в отставку , `` Братья-мусульмане '' не будут стремиться к единоличной власти в стране .", 'Сегодня утром неизвестные расстреляли машину начальника управления информационной политики и пресс-службы президента Дагестана Гаруна Курбанова .', 'Около 22:50 мск двое неизвестных из автомата и пистолета обстреляли автомашину ВАЗ-2106 участкового в поселке Манас Карабудахкентского района Дагестана .', 'По данным следствия , Ю.Привалов , являясь руководителем одной из дочерних компаний ОАО "Совкомфлот" , в период с 2000 по 2005гг .', 'совместно с другими участниками преступной организации совершил хищение денежных средств дочерних компаний ОАО "Совкомфлот" , а также ОАО "Новороссийское морское пароходство" (ОАО "Новошип" ) и его дочерних компаний .', 'Деньги при

In [73]:
from deeppavlov.models.tokenizers.ru_tokenizer import RussianTokenizer
from deeppavlov.models.tokenizers.nltk_tokenizer import NLTKTokenizer
from nltk.corpus import stopwords

russian_stopwords = stopwords.words("russian")
tokenizer = NLTKTokenizer(lowercase = False, stopwords=russian_stopwords)

import string

def tokenize_text(text: str) -> list:
        """
        Tokenize text
        :param text: text for tokenization
        :return: tokens
        """
        tokens = tokenizer([text])[0]
        
        tokens = [t for t in tokens if t not in string.punctuation and t not in russian_stopwords and t.isalpha()]
        return tokens
    


In [74]:

tokenized_sentences = []
for s in sentences:
    tokenized_sentences.append(tokenize_text(s))

    
print(tokenized_sentences)

[['Премьер', 'министр', 'Японии', 'Наото', 'Кан', 'огласил', 'новый', 'состав', 'правительства', 'страны'], ['Так', 'бывшего', 'генерального', 'секретаря', 'правительства', 'Йошито', 'Сенгоку', 'сменил', 'посту', 'Юкио', 'Эдано'], ['Он', 'также', 'заверил', 'случае', 'Мубарак', 'уйдет', 'отставку', 'Братья', 'мусульмане', 'будут', 'стремиться', 'единоличной', 'власти', 'стране'], ['Сегодня', 'утром', 'неизвестные', 'расстреляли', 'машину', 'начальника', 'управления', 'информационной', 'политики', 'пресс', 'службы', 'президента', 'Дагестана', 'Гаруна', 'Курбанова'], ['Около', 'мск', 'двое', 'неизвестных', 'автомата', 'пистолета', 'обстреляли', 'автомашину', 'ВАЗ', 'участкового', 'поселке', 'Манас', 'Карабудахкентского', 'района', 'Дагестана'], ['По', 'данным', 'следствия', 'Ю', 'Привалов', 'являясь', 'руководителем', 'одной', 'дочерних', 'компаний', 'ОАО', 'Совкомфлот', 'период'], ['совместно', 'другими', 'участниками', 'преступной', 'организации', 'совершил', 'хищение', 'денежных', 'ср

In [33]:
index = dict()
all_tokens = set()
for i, tokenized_sentence in enumerate(tokenized_sentences):
    index[i] = dict()
    for token in tokenized_sentence:
        if token in index[i]:
            index[i][token] += 1
        else:
            index[i][token] = 1
        all_tokens.add(token)

# This code I wrote for computing weights of tokens in sentence,
# but in my opinion best text summarization receive only by tf values
#
# # Computing idf values for tokens in article
token_idf = dict()

for token in all_tokens:
    token_idf[token] = 0

for token in token_idf:
    for sentence_number in index:
        if token in index[sentence_number]:
            token_idf[token] += 1

for token in token_idf:
    token_idf[token] = np.log10(float(len(tokenized_sentences)) / token_idf[token])

# Computing matrix for LSA
print(index[0])
# print(token_idf)

{'япония': 1, 'наоть': 1, 'кан': 1, 'огласить': 1, 'новый': 1, 'состав': 1, 'правительство': 1, 'страна': 1}


In [34]:
def query(tokenized_sentence, token_idf):
    word_dict = dict()
    doc_tf_idf_vector = []
    
    for token in tokenized_sentence:
        if token in word_dict:
            word_dict[token] += 1
        else:
            word_dict[token] = 1
        
    for t in tokenized_sentence:
        doc_tf_idf_vector.append(token_idf[t]*word_dict[t]/len(tokenized_sentence))
   
            
    return doc_tf_idf_vector 

In [35]:
a = []
q = []
for s in tokenized_sentences:
    if 'япония' in set(s):
        a.append(query(s, token_idf))
        print(len(s), len(query(s, token_idf)),s)

8 8 ['япония', 'наоть', 'кан', 'огласить', 'новый', 'состав', 'правительство', 'страна']
4 4 ['япония', 'избранный', 'ёсихико', 'нода']
10 10 ['нода', 'стать', 'япония', 'последний', 'год', 'прошедшее', 'время', 'приход', 'власть', 'демократ']
6 6 ['япония', 'подтвердить', 'свой', 'намерение', 'уйти', 'отставка']
7 7 ['уволиться', 'скандал', 'глава', 'мид', 'япония', 'найти', 'замена']
26 26 ['маэхара', 'провести', 'свой', 'пост', 'около', 'полгода', 'вынужденный', 'подать', 'отставка', 'выясниться', 'политический', 'фонд', 'принять', 'пожертвование', 'сумма', 'тысяча', 'иена', 'примерно', 'тысяча', 'доллар', 'жительница', 'япония', 'который', 'иметь', 'японский', 'гражданство']
10 10 ['кроме', 'президент', 'освободить', 'занимать', 'должность', 'российский', 'посол', 'япония', 'михаил', 'белый']
5 5 ['новый', 'япония', 'избранный', 'йосихико', 'нода']
11 11 ['обойти', 'второе', 'тур', 'голосование', 'министр', 'экономика', 'торговля', 'промышленность', 'япония', 'банри', 'кайэда']
20 

In [36]:
# import gensim.downloader as api

# # for k in api.info()['models']:
# #     print(k)
    

In [37]:
# from gensim.models.wrappers import FastText

# model = FastText.load_fasttext_format('../data/Embedders/Fasttext/ft_native_300_ru_wiki_lenta_remstopwords.bin')

In [55]:



for v, s in zip(a, tokenized_sentences):
    
    print(v)
    
    
lens = [len(s) for s in tokenized_sentences]
print('\n', np.max(lens))    



[0.31420213422348436, 0.43920213422348436, 0.4015733847654867, 0.4015733847654867, 0.17413039282275314, 0.23501857727603603, 0.15669321072535, 0.19432196018334766]
[0.6284042684469687, 0.516614760611414, 0.7591239547670531, 0.6671297584434045]
[0.2668519033773618, 0.11139433523068368, 0.2513617073787875, 0.18234209937593612, 0.08761273442753642, 0.2610527086795931, 0.14643990511176935, 0.2610527086795931, 0.18234209937593612, 0.2513617073787875]
[0.4189361789646458, 0.40573930462337504, 0.20430255730960814, 0.40573930462337504, 0.282345523041001, 0.19421870178995237]
[0.4337851170097446, 0.3022395807308339, 0.1472490700255887, 0.3477765468200357, 0.35908815339826783, 0.3428105316401483, 0.3477765468200357]
[0.13513911822261057, 0.08008666575495721, 0.04714674399452496, 0.038476902835382146, 0.0888268111973827, 0.13513911822261057, 0.10521022397708582, 0.06913898962127214, 0.04481970041306593, 0.11678830073339277, 0.07231340839262647, 0.08428452996361369, 0.06236624888836133, 0.13513911